In [10]:
#Importing basics
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import scipy
from scipy.optimize import curve_fit

#Importing my functions
import sys
sys.path.append('/home/c2032014/py_files')
import load_and_clean as lac
import fit_rms_phase as frp
import F_test as ft
import chi_square as chis
import get_obs_file_triplets as g
import get_obs_file_triplets as gft
import G_span_abs as gs
#Importing parallel processing packages
from joblib import Parallel, delayed
import os
import glob
def get_obs_file_pairs(obs_folder, obs_names):
    """
    Collects file1, file2, and gti paths for each observation.
    
    Args:
        obs_folder (str): Path to the folder containing observations.
        obs_names (list): List of observation directory names.
        file1_suffix (str): Filename suffix for file1.
        file2_suffix (str): Filename suffix for file2.
        gti_suffix (str): Filename suffix for GTI file.

    Returns:
        list of tuples: [(file1, file2, gti), ...]
    """
    obs_triplets = []
    for obs in obs_names:
        full_dir = os.path.join(obs_folder, obs)
        gti_path  = os.path.join(full_dir,  f"GTI_ixpe{obs}_evt2_v0*.txt")
        gti = glob.glob(gti_path)[0]
        full_dir = os.path.join(full_dir, "event_1pt5")
        file1_path = os.path.join(full_dir,  f"ixpe{obs}_det12*.fits")
        #print(file1_path)
        file1 = glob.glob(file1_path)[0]
        file2_path = os.path.join(full_dir, f"ixpe{obs}_det3*.fits")
        file2 = glob.glob(file2_path)[0]
        obs_triplets.append((file1, file2, gti))
    return obs_triplets



In [4]:
import numpy as np
norm='abs'

f_bin_number=20
Pmin=51
Pmax=200
gti_obs1='/home/c2032014/CYGX1/event_1pt5/GTI_ixpe01002901_evt1pt5_v01_src.txt'
gti_obs2='/home/c2032014/cyg_june/CYGX1_JUNE/event_1pt5/GTI_ixpe01250101_evt1pt5_v01_src.txt'
bin_length=1/64
seg_length=16
fmin=0
fmax=4
f_bin_number=20
mod_min=np.radians(-90)
mod_max=np.radians(90)
mod_bin_number=20
J=mod_bin_number
spur_sub=True
coherence_corrector=False
output_file='/home/c2032014/cygx1_paper/stacked_q_u_nu_try1.npy'

In [40]:
def process_obs_triplet(obs, Pmin, Pmax, bin_length,
                        seg_length, fmin, fmax,spur_sub,
                        norm,mod_bin_number,mod_min,mod_max,
                        coherence_corrector=None):
    


    #print(obs)
    data1, *_ = lac.load_and_clean(obs[0], Pmin, Pmax)
    data2, *_ = lac.load_and_clean(obs[1], Pmin, Pmax)
    GTI = list(np.loadtxt(obs[2]))

    I_obs = len(data1['TIME'])
    Q_obs = data1['Q']
    U_obs = data1['U']

    Q_norm = np.sum(Q_obs) / I_obs
    U_norm = np.sum(U_obs) / I_obs

    #Calculating the scale factor from deadtime nuances
    scale_factor=(I_obs+len(data2['TIME']))**2/(I_obs* len(data2['TIME']))
    #print('scale factor', scale_factor) 



    lc1 = Lightcurve.make_lightcurve(data1['TIME'], dt=bin_length, gti=GTI)
    lc2 = Lightcurve.make_lightcurve(data2['TIME'], dt=bin_length, gti=GTI)
    lc1.apply_gtis()
    lc2.apply_gtis()

    # cs ref needed for null calculation
    cs_ref = AveragedCrossspectrum.from_lightcurve(lc1, lc2, seg_length, norm='abs')
    ps_2_ref = Powerspectrum.from_lightcurve(lc2, seg_length, norm='abs')

    plt.figure()
    plt.plot(cs_ref.freq, cs_ref.power.real, label='Real part')
  
    aspace = np.linspace(mod_min, mod_max, mod_bin_number + 1)
    mod_min_array = aspace[:-1]
    mod_max_array = aspace[1:]
    av_mod = (mod_min_array + mod_max_array) / 2
    av_mod_err = (mod_max_array - mod_min_array) / 2
   
    G_real_span, G_im_span, n_span, m_span,lc_1_sub_span,lc_spur,cs,spur_sub_norm = gs.G_span(mod_bin_number, data1, lc2, GTI, bin_length, seg_length, fmin, fmax, spur_sub, norm)

    print('new obs cs 0', cs[0])
    print('weights', cs[0].m)


    return {
        "obs_name": obs,
        "I_obs": I_obs,
        "Q_obs": Q_obs,
        "U_obs": U_obs,
        "Q_norm": Q_norm,
        "U_norm": U_norm,
        "cs_ref": cs_ref,
        "cs_G": cs,
        "weights_ref" : cs_ref.m,
        "weights_G": [cs.m for cs in cs],
        "ps_2_ref": ps_2_ref,
        "scale factor": scale_factor,
        
        
        



    }


#Define sinusoidal models to fit Re[G] and Im[G] 
def cross_spec_model_real(phi,A,B,C,J):
    Re_G=(1/J) * ( A + (B*np.cos(2*phi)) + (C*np.sin(2*phi)) )
    return Re_G

#The imaginary sinusoid does not have the A term
def cross_spec_model_imag(phi,B,C,J):
    Im_G=(1/J) * ( (B*np.cos(2*phi)) + (C*np.sin(2*phi)) )
    return Im_G

def cross_spec_model_null(phi, C_nu_mag_sqrd,Q_norm,U_norm,J):
    return (1/J) * C_nu_mag_sqrd * (1 + Q_norm * np.cos(2 * phi) + U_norm * np.sin(2 * phi))



obs_dir='/home/c2032014/cygx1_paper/cyg_data_all'
obs_names=['01002901','01250101','03002201','03003101','03010001','03010101']
obs_triplets=get_obs_file_pairs(obs_dir, obs_names)


#Importing stingray packages
from stingray import Lightcurve, Powerspectrum, AveragedCrossspectrum


In [41]:
    
    
results = Parallel(n_jobs=-1)(delayed(process_obs_triplet)(
        obs, Pmin, Pmax, bin_length, seg_length, fmin, fmax,
        spur_sub, norm, mod_bin_number,mod_min,mod_max,
        coherence_corrector=False
    ) for obs in obs_triplets)

num of events 1365526
num of events 1813589
num of events 1777702
num of events 3883959
num of events 1701018
num of events 632960
num of events 9678551
num of events 823553
num of events 842912
num of events 785926
num of events 1807055
num of events 4450960
new obs cs 0 AveragedCrossspectrum
_____________________
freq           : [ 0.0625 ... 31.9375] (size 511)
power          : [0.8956525 +0.02323708j ... 0.07717621-0.00161126j] (size 511)
power_err      : [0.07035994+0.07035994j ... 0.05478312+0.05478312j] (size 511)
unnorm_power   : [7.16521997+0.18589663j ... 0.61740966-0.01289004j] (size 511)
unnorm_power_err: [0.56287951+0.56287951j ... 0.43826495+0.43826495j] (size 511)
countrate1     : 0.7423157212784336
countrate2     : 7.160829974085805
df             : 0.0625
dt             : 0.015625
err_dist       : poisson
fullspec       : False
gti            : [[2.35152764e+08 2.35152828e+08]
 ...
 [2.35250763e+08 2.35254296e+08]] (shape (145, 2))
k              : 1
m              : 3

In [16]:
#obs_triplets = gft.get_obs_file_pairs(obs_folder, obs_names)
    # Parallel processing of obs_triplets
    


#print(results)

aspace = np.linspace(mod_min, mod_max, mod_bin_number + 1)
mod_min_array = aspace[:-1]
mod_max_array = aspace[1:]
av_mod = (mod_min_array + mod_max_array) / 2
av_mod_err = (mod_max_array - mod_min_array) / 2

    #Making frequency list
fspace = np.linspace(fmin, fmax, f_bin_number + 1)
f_min_array, f_max_array = fspace[:-1], fspace[1:]
f_angle_list = list(zip(f_min_array, f_max_array))
av_f = (f_min_array + f_max_array) / 2
av_f_err = (f_max_array - f_min_array) / 2


# Unpack the results
I_obs_arr = [r["I_obs"] for r in results]
Q_obs_arr = [r["Q_obs"] for r in results]
U_obs_arr = [r["U_obs"] for r in results]
cs_ref_obs = [r["cs_ref"] for r in results]
cs_G_obs = [r["cs_G"] for r in results]
weights_ref_obs = [r["weights_ref"] for r in results]
weights_G_obs = [r["weights_G"] for r in results]
ps_2_ref = [r["ps_2_ref"] for r in results]
scale_factors = [r["scale factor"] for r in results]
obs_names = [r["obs_name"] for r in results]
G_real_span = [r["G_real_span"] for r in results]
G_im_span = [r["G_im_span"] for r in results]

#print('obs_names', obs_names)
#print('scale_factors', scale_factors)
cs_G_obs_real = [[cs.power.real for cs in sublist] for sublist in cs_G_obs]
cs_G_obs_imag = [[cs.power.imag for cs in sublist] for sublist in cs_G_obs]

#print('cs_G_obs_real',cs_G_obs_real)

#print('cs_G_obs',cs_G_obs)

cs_G_obs_real=np.array(cs_G_obs_real)*np.array( scale_factors)[:, None, None]
cs_G_obs_imag=np.array(cs_G_obs_imag)*np.array( scale_factors)[:, None, None]
#cs_G_obs_real=scale_factors* np.array(cs_G_obs_real)
#print('cs_G_obs_real scale',cs_G_obs_real)

cs_ref_obs_real = [cs_ref.power.real for cs_ref in cs_ref_obs]
cs_ref_obs_imag = [cs_ref.power.imag for cs_ref in cs_ref_obs]
cs_ref_freqs = [cs_ref.freq for cs_ref in cs_ref_obs]



I_tot=np.sum(I_obs_arr)
Q_norm=sum(sum(sublist) for sublist in Q_obs_arr)/I_tot
U_norm=sum(sum(sublist) for sublist in U_obs_arr)/I_tot


weights_expanded_obs = np.array(weights_G_obs)[:, :, np.newaxis]


# Weighted sum over axis=0
numerator_obs_real = np.sum(cs_G_obs_real * weights_expanded_obs, axis=0)       # (20, 511)
denominator_obs_real = np.sum(weights_expanded_obs, axis=0)                     # (20, 1)

numerator_obs_imag = np.sum(cs_G_obs_imag * weights_expanded_obs, axis=0)       # (20, 511)
denominator_obs_imag = np.sum(weights_expanded_obs, axis=0)                     # (20, 1)



# Final weighted average
cs_G_real_stacked = numerator_obs_real / np.where(denominator_obs_real == 0, np.nan, denominator_obs_real)
cs_G_imag_stacked = numerator_obs_imag / np.where(denominator_obs_imag == 0, np.nan, denominator_obs_imag)
#print('weights_G_obs',weights_G_obs)
print('cs_G_real_stacked 0',cs_G_real_stacked[0])

cs_ref_real_stacked=np.average(cs_ref_obs_real,weights=weights_ref_obs,axis=0)
cs_ref_imag_stacked=np.average(cs_ref_obs_imag,weights=weights_ref_obs,axis=0)
print('cs_ref_real_stacked ',cs_ref_real_stacked)
print('shape ref stacked',cs_ref_real_stacked.shape)


num of events 1365526
num of events 1777702
num of events 1813589
num of events 1701018
num of events 3883959
num of events 632960
num of events 823553
num of events 842912
num of events 785926
num of events 9678551
num of events 1807055
num of events 4450960
new obs cs 0 AveragedCrossspectrum
_____________________
freq           : [ 0.0625 ... 31.9375] (size 511)
power          : [0.8956525 +0.02323708j ... 0.07717621-0.00161126j] (size 511)
power_err      : [0.07035994+0.07035994j ... 0.05478312+0.05478312j] (size 511)
unnorm_power   : [7.16521997+0.18589663j ... 0.61740966-0.01289004j] (size 511)
unnorm_power_err: [0.56287951+0.56287951j ... 0.43826495+0.43826495j] (size 511)
countrate1     : 0.7423157212784336
countrate2     : 7.160829974085805
df             : 0.0625
dt             : 0.015625
err_dist       : poisson
fullspec       : False
gti            : [[2.35152764e+08 2.35152828e+08]
 ...
 [2.35250763e+08 2.35254296e+08]] (shape (145, 2))
k              : 1
m              : 3

In [27]:
#plot the individual cross spectra with the stacked one
plt.figure(figsize=(12, 8))
plt.plot(cs_G_real_stacked,cs_G_obs[0].freq)



AttributeError: 'tuple' object has no attribute 'freq'

<Figure size 864x576 with 0 Axes>

In [22]:
results_freq = Parallel(n_jobs=-1)(delayed(process_frequency_bin)(i,cs_G_real_stacked,cs_G_imag_stacked,cs_ref_real_stacked,cs_ref_imag_stacked,cs_G_obs[0][0].freq,av_mod,Q_norm,U_norm) for i in (f_angle_list))


parameters_realparameters_real  parameters_real parameters_real parameters_real [ 6.47059036 -0.5726074   0.56908796]
[ 4.95566738 -0.1469132  -0.00604266]
[20.41576524  0.47964669  0.70336045]
[164.33655836   1.46399132  -0.40450538]
[ 4.73841072 -0.13953648  0.24967561]
parameters_real parameters_real [ 8.94152052 -0.16476395  0.61797131]
[16.92356978  0.36146487  0.41321327]
parameters_real parameters_real [24.30386538 -0.06421355 -0.47284783]
parameters_real [28.43963283  0.44421323 -0.17876804]
parameters_realparameters_real  parameters_real [11.97541458 -0.03784287  0.18492712]
parameters_real parameters_real[35.68845453  0.64361475 -0.22180849] 
[8.0574291  1.11768637 0.07810261]parameters_real
 [10.3533791  -0.3426437  -0.20197291]
[8.08377963e+01 2.43725407e-01 5.64418341e-02]
[ 3.87928432 -0.47264054 -0.16578209]
[48.48969302  0.54109451  0.19289108]
parameters_real [ 6.28736296 -0.36083615  0.371455  ]
parameters_real [5.7537096  0.21879117 0.11366699]
parameters_real parame

In [21]:
def process_frequency_bin(i,cs_G_real_stacked,cs_G_imag_stacked,cs_ref_real_stacked,cs_ref_imag_stacked,cs_freqs,av_mod,Q_norm,U_norm):

    G_real=[cs_G_real_stacked[(i[0]<=cs_freqs) & (cs_freqs<=i[1])].mean() for cs_G_real_stacked in cs_G_real_stacked]
    G_imag=[cs_G_imag_stacked[(i[0]<=cs_freqs) & (cs_freqs<=i[1])].mean() for cs_G_imag_stacked in cs_G_imag_stacked]

    cs_ref_real_mean=np.array(cs_ref_real_stacked[(i[0] <= cs_freqs) & (cs_freqs <= i[1])].mean())
    cs_ref_imag_mean=np.array(cs_ref_imag_stacked[(i[0] <= cs_freqs) & (cs_freqs <= i[1])].mean())
  
    cs_ref_average=cs_ref_real_mean+1j*cs_ref_imag_mean
    cs_ref_abs_mean_stack=np.abs(cs_ref_average)
    J=20
    def cross_spec_model_real_fixedJ(phi, A, B, C):
        return cross_spec_model_real(phi, A, B, C, J=J)
    
    def cross_spec_model_imag_fixedJ(phi, B, C):
        return cross_spec_model_imag(phi, B, C, J=J)
    

    #Fit real and im parts of G
    parameters_real,pcovreal=curve_fit(cross_spec_model_real_fixedJ,np.array(av_mod),np.array(G_real))
    print('parameters_real',parameters_real)

    parameters_imag,pcovimag=curve_fit(cross_spec_model_imag_fixedJ,np.array(av_mod),np.array(G_imag))
    
    #fit_imag = cross_spec_model_imag(np.array(av_mod),parameters_imag[0],parameters_imag[1])
    A_real, B_real, C_real = parameters_real
    Areal_err,Breal_err,Creal_err= np.sqrt(np.diag(pcovreal))
   
    Bimag_err,Cimag_err= np.sqrt(np.diag(pcovimag))
   
    def cross_spec_model_null_fixedJ(phi, C_nu_mag_sqrd, Q_norm, U_norm):
        return cross_spec_model_null(phi, C_nu_mag_sqrd, Q_norm, U_norm, J=J)

    #Calculating null hypothesis
    G_null=cross_spec_model_null_fixedJ(np.array(av_mod),cs_ref_abs_mean_stack,Q_norm,U_norm)
    G_null_real=G_null.real
    G_null_imag=G_null.imag
    
    #Null hypothesis co-efficients
    A_null_real=cs_ref_abs_mean_stack
    B_null_real=cs_ref_abs_mean_stack*Q_norm
    C_null_real=cs_ref_abs_mean_stack*U_norm
    B_null_imag=0
    C_null_imag=0
 
    f_av = (i[0] + i[1]) / 2

# [All code currently in your loop goes here, unmodified]
# Replace `.append()` calls with return values instead

    result = {
    "f_av": f_av,
    "A_real_err": Areal_err,
    "B_real_err": Breal_err,
    "C_real_err": Creal_err,
    "B_imag_err": Bimag_err,
    "C_imag_err": Cimag_err,
    "A_real": A_real,
    "B_real": B_real,
    "C_real": C_real,
    "B_imag": parameters_imag[0],
    "C_imag": parameters_imag[1],
    "A_null_real": A_null_real,
    "B_null_real": B_null_real,
    "C_null_real": C_null_real,
    "B_null_imag": B_null_imag,
    "C_null_imag": C_null_imag,
    "cs_G_real_stacked": cs_G_real_stacked,
    "cs_G_imag_stacked": cs_G_imag_stacked,
    "cs_ref_real_stacked": cs_ref_real_stacked,
    "cs_ref_imag_stacked": cs_ref_imag_stacked,
    "G_real": G_real,
    "G_imag": G_imag,
    "G_null_real": G_null_real,
    "G_null_imag": G_null_imag,
    

    }   

    return result